# Using structural connections to build common spaces
This cell contains the name of the title that will be linked from the previous page 
This notebook or markdown file depending on how you want to do this Shaun is the chapter space for your material


In [ ]:
from repo2data.repo2data import Repo2Data
import os
# # install the data if running locally, or points to cached data if running on neurolibre
data_req_path = os.path.join("../../", "binder", "data_requirement.json")
# # download data
repo2data = Repo2Data(data_req_path,)
data_path = repo2data.install()
data_dir=os.path.realpath('../../data/Xspecies/googledrive/03-warrington/')

print(f'data for this chapter is stored in the {data_dir}')
### copy any pngs in the data folder to this notebook folder 
!mkdir -p images
!cp {data_dir}/*png images/

Place whatever you want here

Diffusion tractography allows for the reconstruction of major white matter fibre bundles, many of which are preserved across evolution and show cortical connectivity patterns common and/or divergent across species. These (dis)similarities may be probed to understand brain evolution. In this talk, I will introduce how we can build common spaces anchored on major white matter fibre bundles to quantitatively compare brains across evolution. 


# Example header DWI here


enter text and such here

enter text and such here 

here is your example AI generated image showing the data loading works 

pngs can be placed with all the data. but if using markdown to show them then the path is images/<<imagename.png>>

please use pngs for now 


![example image](images/ex_image.png "example image")

# Xtract is awesome

blah blah blah here are some examles of how amazing extract is 

a scatter plot -- anything you have will be executed and saved statically in the book just in case 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate random data
np.random.seed(0)
x = np.random.rand(100)
y = np.random.rand(100)

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='r', alpha=0.5, label='Random Data')
plt.title('Scatter Plot of Random Data')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.grid(True)
plt.show()
